# Accessing Global Height data from Microsoft Planetary Computer

Set-up the pystac client to access the Microsoft Planetary Computer catalog

In [ ]:
from pystac_client import Client
import planetary_computer

catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,
)

Define collection and assets to retrieve and construct [STAC API filters](https://github.com/stac-api-extensions/filter) for efficient query performance against Planetary Computer API

In [ ]:
collections = ["met-office-global-deterministic-height"]
asset_id = "cloud_amount_on_height_levels"
datacube_extension_filters = {
    "op": "and",
    "args": [
        {
            "op": "=",
            "args": [ { "property": "forecast:reference_datetime" }, "2026-01-14T12:00:00Z" ]
        },
        {
            "op": "=",
            "args": [ { "property": "forecast:horizon" }, "PT0144H00M" ]
        }
    ]
}

Search Planetary Computer catalog for STAC items and retrieve STAC Asset URL

In [ ]:
search = catalog.search(
    collections=collections,
    filter_lang= "cql2-json",
    filter=datacube_extension_filters
)

items = search.item_collection()
asset_url = items.items[0].assets[asset_id].href

Example usage: Plot NetCDF data

In [ ]:
import fsspec
import xarray as xr
import matplotlib.pyplot as plt

example_netcdf = xr.open_dataset(fsspec.open(asset_url, expand=True).open())
plt.figure(figsize=(10, 5))
example_netcdf["cloud_volume_fraction_in_atmosphere_layer"].plot()